# MNIST
Here we will be using keras for digit recognition. We will be using MNIST as our training data for our neural network and then we will test the data and display the results. 

We will be using numpy to get and store the bytes in an array then we will be using keras to train our neural network.

First we take the files taken from:(http://yann.lecun.com/exdb/mnist/) using gzip which is a module that provides open(), compress() and decompress() convenience functions. The GzipFile class reads and writes gzip-format files, automatically compressing or decompressing the data so that it looks like an ordinary file object.

Using numpy we convert the files to a 2D array then reshape it to 28X28 every 784 bytes after the first 16 as unsigned intergers using np.uint8.

In [1]:
# For unzipping the file within the script.
import gzip
with gzip.open('data/t10k-images-idx3-ubyte.gz', 'rb') as f:
    file_content_images = f.read()
    
# For unzipping the file within the script.
with gzip.open('data/t10k-labels-idx1-ubyte.gz', 'rb') as f:
    file_content_labels = f.read()

import numpy as np
image = ~np.array(list(file_content_images[16:800])).reshape(28,28).astype(np.uint8)

with gzip.open('data/train-images-idx3-ubyte.gz', 'rb') as f:
    train_img = f.read()

with gzip.open('data/train-labels-idx1-ubyte.gz', 'rb') as f:
    train_lbl = f.read()
    
train_img = ~np.array(list(train_img[16:])).reshape(60000, 28, 28).astype(np.uint8)/255.0
train_lbl =  np.array(list(train_lbl[ 8:])).astype(np.uint8)

# scikit-learn
Scikit-learn (formerly scikits.learn) is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.

In [2]:
# For encoding categorical variables and pre processing.
import sklearn.preprocessing as pre

encoder = pre.LabelBinarizer()
encoder.fit(train_lbl)
outputs = encoder.transform(train_lbl)

outputs[0]

inputs = train_img.reshape(60000, 784)

# Model
There are 6 hidden layers made with a dropout layer running sequentially.

The model takes in the 28X28 array of 784 where each pixel is a number ranging from 0 to 1 where 0 is black and 1 is white. Each one of these pixels is considered a neuron and passed into the neural network where the greyscale value is is the weight.

![title](https://achintavarna.files.wordpress.com/2017/11/mnist_2layers.png)

# Keras
Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

In [11]:
# ------- MODEL -------
# Import keras.
import keras as kr
# Importing the required Keras modules containing model and layers
from keras.models import Sequential
from keras.layers import Dropout 

# Start a neural network, building it by layers.
model = kr.models.Sequential()
# Add a hidden layer with 750 neurons.
model.add(kr.layers.Dense(units=784, activation='relu', input_dim=784))
# Add a hidden layer with 455 neurons.
model.add(kr.layers.Dense(units=455, activation='relu'))
# Add a hidden layer with 250 neurons.
model.add(kr.layers.Dense(units=250, activation='relu'))
# Add a hidden layer with 170 neurons.
model.add(kr.layers.Dense(units=170, activation='softplus'))
# Add a hidden layer with 120 neurons.
model.add(kr.layers.Dense(units=120, activation='linear'))
# Add a hidden layer with 50 neurons.
model.add(kr.layers.Dense(units=50, activation='relu'))
# Add a dropout layer every 1 in 5.
model.add(Dropout(0.2))

# Add a three neuron output layer.
model.add(kr.layers.Dense(units=10, activation='softmax'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_9 (Dense)              (None, 455)               357175    
_________________________________________________________________
dense_10 (Dense)             (None, 250)               114000    
_________________________________________________________________
dense_11 (Dense)             (None, 170)               42670     
_________________________________________________________________
dense_12 (Dense)             (None, 120)               20520     
_________________________________________________________________
dense_13 (Dense)             (None, 50)                6050      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
__________

In [9]:
# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Number of Epoch is the amount of times the training set is put through the model
# The batch size is the amount of images the models processes at one time
model.fit(inputs, outputs, epochs=8, batch_size=100)

with gzip.open('data/t10k-images-idx3-ubyte.gz', 'rb') as f:
    test_img = f.read()

with gzip.open('data/t10k-labels-idx1-ubyte.gz', 'rb') as f:
    test_lbl = f.read()
    
test_img = ~np.array(list(test_img[16:])).reshape(10000, 784).astype(np.uint8) / 255.0
test_lbl =  np.array(list(test_lbl[ 8:])).astype(np.uint8)

Epoch 1/8
60000/60000 [==============================] - 23s 385us/step - loss: 0.7181 - acc: 0.7638
Epoch 2/8
60000/60000 [==============================] - 24s 395us/step - loss: 0.3017 - acc: 0.9119
Epoch 3/8
60000/60000 [==============================] - 23s 381us/step - loss: 0.2297 - acc: 0.9315
Epoch 4/8
60000/60000 [==============================] - 23s 380us/step - loss: 0.1906 - acc: 0.9443
Epoch 5/8
60000/60000 [==============================] - 23s 379us/step - loss: 0.1628 - acc: 0.9515
Epoch 6/8
60000/60000 [==============================] - 23s 388us/step - loss: 0.1498 - acc: 0.9551
Epoch 7/8
60000/60000 [==============================] - 23s 387us/step - loss: 0.1345 - acc: 0.9601
Epoch 8/8
60000/60000 [==============================] - 23s 376us/step - loss: 0.1210 - acc: 0.9630


In [10]:
print((encoder.inverse_transform(model.predict(test_img)) == test_lbl).sum())

9654


# References 
Gzip: https://docs.python.org/3/library/gzip.html

scikit-learn: https://scikit-learn.org/stable/modules/preprocessing.html

Keras: https://keras.io/